## 📊 SCRIPT DE CRÉATION DE DATASET SEMI-SUPERVISÉ

---

Ce script est conçu pour générer un jeu de données optimisé pour les modèles d'**apprentissage semi-supervisé (Semi-Supervised Learning)**, en respectant des proportions et un équilibrage stricts.

### 📝 Paramètres Clés du Dataset

| Paramètre | Valeur | Objectif |
| :--- | :--- | :--- |
| **Ratio de Supervision** | **30% / 70%** | Utilisation maximale de données non-labellisées. |
| **Images Labellisées** | 30% | Données de départ pour l'entraînement du modèle. |
| **Images Non-Labellisées** | 70% | Utilisation pour les techniques de *Pseudo-Labeling* ou de cohérence. |
| **Taille des Classes** | **6 844 images** | Assurer un équilibre parfait entre toutes les classes (minimum commun). |

---

### 💡 Note d'Exécution

Pour exécuter la création de ce dataset, veuillez vous assurer que la fonction de sous-échantillonnage (down-sampling) est configurée pour cibler le nombre minimum commun de **6 844 images** par classe pour garantir l'équilibrage.

In [1]:


import os
import random
import shutil
from pathlib import Path
import sys

class SemiSupervisedDatasetCreator:
    def __init__(self):
        self.source_dir = "skintone"  # Votre dossier source
        self.target_dir = "skintone_data"  # Dossier de sortie
        self.random_seed = 42  # Pour reproductibilité
        
        # Paramètres standards de recherche
        self.TOTAL_PER_CLASS = 6844  # Nombre minimum d'images par classe
        self.LABELLED_RATIO = 0.30   # 30% labellisé
        self.UNLABELLED_RATIO = 0.70  # 70% non-labellisé
        
        # Split interne pour données labellisées
        self.TRAIN_RATIO = 0.70      # 70% train
        self.VAL_RATIO = 0.15        # 15% validation
        self.TEST_RATIO = 0.15       # 15% test
        
        # Calcul des quantités
        self.calculate_counts()
        
        # Classes
        self.classes = ['dark', 'light', 'mid-dark', 'mid-light']
        
        random.seed(self.random_seed)
    
    def calculate_counts(self):
        """Calcule les nombres d'images pour chaque split"""
        self.labelled_per_class = int(self.TOTAL_PER_CLASS * self.LABELLED_RATIO)
        self.unlabelled_per_class = self.TOTAL_PER_CLASS - self.labelled_per_class
        
        self.train_per_class = int(self.labelled_per_class * self.TRAIN_RATIO)
        self.val_per_class = int(self.labelled_per_class * self.VAL_RATIO)
        self.test_per_class = self.labelled_per_class - self.train_per_class - self.val_per_class
        
        print("=" * 60)
        print("CONFIGURATION DU DATASET SEMI-SUPERVISÉ")
        print("=" * 60)
        print(f"Images par classe: {self.TOTAL_PER_CLASS:,}")
        print(f"Ratio: {self.LABELLED_RATIO*100:.0f}% labellisé / {self.UNLABELLED_RATIO*100:.0f}% non-labellisé")
        print(f"\nDétails par classe:")
        print(f"  Labellisé: {self.labelled_per_class:,} images")
        print(f"    ├── Train: {self.train_per_class:,} (70%)")
        print(f"    ├── Validation: {self.val_per_class:,} (15%)")
        print(f"    └── Test: {self.test_per_class:,} (15%)")
        print(f"  Non-labellisé: {self.unlabelled_per_class:,} images")
        print("=" * 60)
    
    def verify_source_data(self):
        """Vérifie que les données source existent"""
        print("\n🔍 VÉRIFICATION DES DONNÉES SOURCE")
        
        if not os.path.exists(self.source_dir):
            print(f"❌ ERREUR: Le dossier '{self.source_dir}' n'existe pas!")
            print(f"   Placez ce script dans le même dossier que 'skintone/'")
            return False
        
        # Vérifier chaque classe
        missing_classes = []
        for cls in self.classes:
            class_path = os.path.join(self.source_dir, cls)
            if not os.path.exists(class_path):
                missing_classes.append(cls)
            else:
                # Compter les images
                images = [f for f in os.listdir(class_path) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                print(f"  ✓ {cls}: {len(images):,} images")
                
                # Vérifier si assez d'images
                if len(images) < self.TOTAL_PER_CLASS:
                    print(f"    ⚠️  Attention: {cls} a seulement {len(images)} images")
                    print(f"    Nous allons utiliser {len(images)} au lieu de {self.TOTAL_PER_CLASS}")
        
        if missing_classes:
            print(f"\n❌ Classes manquantes: {missing_classes}")
            return False
        
        return True
    
    def create_directory_structure(self):
        """Crée la structure de dossiers cible"""
        print("\n📁 CRÉATION DE LA STRUCTURE DES DOSSIERS")
        
        # Dossiers principaux
        main_dirs = [
            f"{self.target_dir}/labelled/train",
            f"{self.target_dir}/labelled/val", 
            f"{self.target_dir}/labelled/test",
            f"{self.target_dir}/unlabelled",
            f"{self.target_dir}/metadata"
        ]
        
        # Sous-dossiers par classe
        for split in ['train', 'val', 'test']:
            for cls in self.classes:
                main_dirs.append(f"{self.target_dir}/labelled/{split}/{cls}")
        
        # Création
        for dir_path in main_dirs:
            os.makedirs(dir_path, exist_ok=True)
            print(f"  ✓ {dir_path}")
        
        return True
    
    def process_class(self, cls):
        """Traite une classe spécifique"""
        print(f"\n📊 Traitement de la classe: {cls}")
        
        # Chemin source
        source_path = os.path.join(self.source_dir, cls)
        
        # Lister toutes les images
        all_images = [
            f for f in os.listdir(source_path) 
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        
        # Si moins d'images que prévu, ajuster
        available_count = len(all_images)
        if available_count < self.TOTAL_PER_CLASS:
            print(f"  ⚠️  Seulement {available_count} images disponibles")
            print(f"  Ajustement du plan...")
            
            # Ajuster dynamiquement
            actual_total = available_count
            actual_labelled = int(actual_total * self.LABELLED_RATIO)
            actual_unlabelled = actual_total - actual_labelled
            
            actual_train = int(actual_labelled * self.TRAIN_RATIO)
            actual_val = int(actual_labelled * self.VAL_RATIO)
            actual_test = actual_labelled - actual_train - actual_val
        else:
            # Prendre exactement le nombre requis
            actual_total = self.TOTAL_PER_CLASS
            actual_labelled = self.labelled_per_class
            actual_unlabelled = self.unlabelled_per_class
            actual_train = self.train_per_class
            actual_val = self.val_per_class
            actual_test = self.test_per_class
        
        print(f"  Total à utiliser: {actual_total} images")
        
        # Échantillonnage aléatoire
        selected_images = random.sample(all_images, min(actual_total, len(all_images)))
        random.shuffle(selected_images)
        
        # Split labellisé vs non-labellisé
        labelled_images = selected_images[:actual_labelled]
        unlabelled_images = selected_images[actual_labelled:actual_total]
        
        # Split labellisé en train/val/test
        train_images = labelled_images[:actual_train]
        val_images = labelled_images[actual_train:actual_train + actual_val]
        test_images = labelled_images[actual_train + actual_val:]
        
        # Copier les images labellisées
        stats = {'labelled': 0, 'unlabelled': 0}
        
        # Train
        for img in train_images:
            src = os.path.join(source_path, img)
            dst = os.path.join(self.target_dir, 'labelled', 'train', cls, img)
            shutil.copy2(src, dst)
            stats['labelled'] += 1
        
        # Validation
        for img in val_images:
            src = os.path.join(source_path, img)
            dst = os.path.join(self.target_dir, 'labelled', 'val', cls, img)
            shutil.copy2(src, dst)
            stats['labelled'] += 1
        
        # Test
        for img in test_images:
            src = os.path.join(source_path, img)
            dst = os.path.join(self.target_dir, 'labelled', 'test', cls, img)
            shutil.copy2(src, dst)
            stats['labelled'] += 1
        
        # Copier les images non-labellisées avec nom unique
        unlabelled_count = 0
        unlabelled_dir = os.path.join(self.target_dir, 'unlabelled')
        
        for img in unlabelled_images:
            src = os.path.join(source_path, img)
            
            # Créer un nom unique avec classe d'origine
            img_name, img_ext = os.path.splitext(img)
            unique_name = f"{cls}_{img_name}_{unlabelled_count:06d}{img_ext}"
            dst = os.path.join(unlabelled_dir, unique_name)
            
            shutil.copy2(src, dst)
            stats['unlabelled'] += 1
            unlabelled_count += 1
        
        print(f"  ✓ Labellisé: {stats['labelled']} images (train: {len(train_images)}, val: {len(val_images)}, test: {len(test_images)})")
        print(f"  ✓ Non-labellisé: {stats['unlabelled']} images")
        
        return stats
    
    def create_metadata(self):
        """Crée des fichiers metadata avec les statistiques"""
        print("\n📝 CRÉATION DES FICHIERS MÉTADONNÉES")
        
        metadata_dir = os.path.join(self.target_dir, 'metadata')
        
        # Compter les images dans chaque dossier
        stats = {
            'labelled': {'train': {}, 'val': {}, 'test': {}},
            'unlabelled': 0
        }
        
        # Compter labellisé
        for split in ['train', 'val', 'test']:
            for cls in self.classes:
                split_dir = os.path.join(self.target_dir, 'labelled', split, cls)
                if os.path.exists(split_dir):
                    count = len([f for f in os.listdir(split_dir) 
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    stats['labelled'][split][cls] = count
        
        # Compter non-labellisé
        unlabelled_dir = os.path.join(self.target_dir, 'unlabelled')
        if os.path.exists(unlabelled_dir):
            stats['unlabelled'] = len([f for f in os.listdir(unlabelled_dir) 
                                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        
        # Écrire les statistiques
        with open(os.path.join(metadata_dir, 'statistics.txt'), 'w') as f:
            f.write("=" * 60 + "\n")
            f.write("STATISTIQUES DU DATASET SEMI-SUPERVISÉ\n")
            f.write("=" * 60 + "\n\n")
            
            f.write("CONFIGURATION:\n")
            f.write(f"Source: {self.source_dir}\n")
            f.write(f"Destination: {self.target_dir}\n")
            f.write(f"Ratio: {self.LABELLED_RATIO*100:.0f}% labellisé / {self.UNLABELLED_RATIO*100:.0f}% non-labellisé\n")
            f.write(f"Images par classe (cible): {self.TOTAL_PER_CLASS}\n\n")
            
            f.write("RÉSULTATS PAR CLASSE:\n")
            f.write("-" * 60 + "\n")
            
            # Calculer les totaux
            total_labelled = 0
            total_unlabelled = stats['unlabelled']
            
            for cls in self.classes:
                f.write(f"\n{cls.upper()}:\n")
                
                # Labellisé
                cls_labelled = 0
                for split in ['train', 'val', 'test']:
                    count = stats['labelled'][split].get(cls, 0)
                    cls_labelled += count
                    f.write(f"  {split}: {count} images\n")
                
                total_labelled += cls_labelled
                f.write(f"  Total labellisé: {cls_labelled}\n")
            
            f.write("\n" + "=" * 60 + "\n")
            f.write("TOTAUX GÉNÉRAUX:\n")
            f.write(f"  Images labellisées: {total_labelled}\n")
            f.write(f"  Images non-labellisées: {total_unlabelled}\n")
            f.write(f"  TOTAL: {total_labelled + total_unlabelled}\n\n")
            
            f.write("RATIOS FINAUX:\n")
            total_all = total_labelled + total_unlabelled
            if total_all > 0:
                f.write(f"  Labellisé: {(total_labelled/total_all*100):.1f}%\n")
                f.write(f"  Non-labellisé: {(total_unlabelled/total_all*100):.1f}%\n")
        
        print(f"  ✓ statistics.txt créé dans {metadata_dir}")
        
        # Créer un fichier CSV pour le mapping non-labellisé
        with open(os.path.join(metadata_dir, 'unlabelled_mapping.csv'), 'w') as f:
            f.write("filename,original_class\n")
            
            unlabelled_dir = os.path.join(self.target_dir, 'unlabelled')
            if os.path.exists(unlabelled_dir):
                for filename in os.listdir(unlabelled_dir):
                    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                        # Extraire la classe d'origine du nom de fichier
                        parts = filename.split('_')
                        if len(parts) >= 1:
                            original_class = parts[0]
                            f.write(f"{filename},{original_class}\n")
        
        print(f"  ✓ unlabelled_mapping.csv créé")
    
    def run(self):
        """Exécute le pipeline complet"""
        print("🚀 DÉBUT DE LA CRÉATION DU DATASET SEMI-SUPERVISÉ")
        print("=" * 60)
        
        # Étape 1: Vérification
        if not self.verify_source_data():
            print("\n❌ Impossible de continuer. Vérifiez vos données source.")
            return False
        
        # Étape 2: Création structure
        self.create_directory_structure()
        
        # Étape 3: Traitement de chaque classe
        print("\n⚙️  TRAITEMENT DES CLASSES")
        print("-" * 60)
        
        all_stats = {}
        for cls in self.classes:
            all_stats[cls] = self.process_class(cls)
        
        # Étape 4: Metadata
        self.create_metadata()
        
        # Étape 5: Résumé final
        self.print_summary()
        
        return True
    
    def print_summary(self):
        """Affiche un résumé final"""
        print("\n" + "=" * 60)
        print("✅ DATASET CRÉÉ AVEC SUCCÈS!")
        print("=" * 60)
        
        # Calcul des totaux
        total_labelled = 0
        total_unlabelled = 0
        
        for split in ['train', 'val', 'test']:
            split_dir = os.path.join(self.target_dir, 'labelled', split)
            for cls in self.classes:
                class_dir = os.path.join(split_dir, cls)
                if os.path.exists(class_dir):
                    count = len([f for f in os.listdir(class_dir) 
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    total_labelled += count
        
        unlabelled_dir = os.path.join(self.target_dir, 'unlabelled')
        if os.path.exists(unlabelled_dir):
            total_unlabelled = len([f for f in os.listdir(unlabelled_dir) 
                                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        
        total_all = total_labelled + total_unlabelled
        
        print(f"\n📊 STATISTIQUES FINALES:")
        print(f"  Images labellisées: {total_labelled:,}")
        print(f"  Images non-labellisées: {total_unlabelled:,}")
        print(f"  TOTAL: {total_all:,}")
        
        if total_all > 0:
            print(f"\n📈 RATIOS FINAUX:")
            print(f"  Labellisé: {(total_labelled/total_all*100):.1f}% (objectif: 30%)")
            print(f"  Non-labellisé: {(total_unlabelled/total_all*100):.1f}% (objectif: 70%)")
        
        print(f"\n📁 STRUCTURE CRÉÉE:")
        print(f"  {self.target_dir}/")
        print(f"    ├── labelled/")
        print(f"    │   ├── train/ (images organisées par classe)")
        print(f"    │   ├── val/   (images organisées par classe)")
        print(f"    │   └── test/  (images organisées par classe)")
        print(f"    ├── unlabelled/ (toutes images mélangées)")
        print(f"    └── metadata/ (fichiers statistiques)")
        
        print(f"\n✨ Votre dataset est prêt pour:")
        print(f"   - Entraînement supervisé: utiliser 'labelled/train/'")
        print(f"   - Méthodes semi-supervisées: utiliser 'labelled/train/' + 'unlabelled/'")
        print(f"   - Évaluation: utiliser 'labelled/test/'")


def main():
    """Fonction principale"""
    print("🎯 CRÉATEUR DE DATASET SEMI-SUPERVISÉ")
    print("Ratio: 30% labellisé / 70% non-labellisé")
    print("=" * 60)
    
    # Créer l'instance
    creator = SemiSupervisedDatasetCreator()
    
    # Exécuter
    success = creator.run()
    
    if success:
        print("\n" + "=" * 60)
        print("✅ Opération terminée avec succès!")
        print(f"📁 Dataset disponible dans: {creator.target_dir}")
        print("=" * 60)
    else:
        print("\n❌ Échec de la création du dataset.")
        sys.exit(1)


if __name__ == "__main__":
    """
    INSTRUCTIONS:
    1. Placez ce script dans le même dossier que 'skintone/'
    2. Exécutez: python create_semi_supervised_dataset.py
    3. Attendez la fin du traitement
    
    STRUCTURE ATTENDUE EN ENTRÉE:
    skintone/
        ├── dark/           (images .jpg/.png)
        ├── light/          (images .jpg/.png)
        ├── mid-dark/       (images .jpg/.png)
        └── mid-light/      (images .jpg/.png)
    """
    main()

🎯 CRÉATEUR DE DATASET SEMI-SUPERVISÉ
Ratio: 30% labellisé / 70% non-labellisé
CONFIGURATION DU DATASET SEMI-SUPERVISÉ
Images par classe: 6,844
Ratio: 30% labellisé / 70% non-labellisé

Détails par classe:
  Labellisé: 2,053 images
    ├── Train: 1,437 (70%)
    ├── Validation: 307 (15%)
    └── Test: 309 (15%)
  Non-labellisé: 4,791 images
🚀 DÉBUT DE LA CRÉATION DU DATASET SEMI-SUPERVISÉ

🔍 VÉRIFICATION DES DONNÉES SOURCE
  ✓ dark: 8,640 images
  ✓ light: 9,769 images
  ✓ mid-dark: 10,576 images
  ✓ mid-light: 6,844 images

📁 CRÉATION DE LA STRUCTURE DES DOSSIERS
  ✓ skintone_data/labelled/train
  ✓ skintone_data/labelled/val
  ✓ skintone_data/labelled/test
  ✓ skintone_data/unlabelled
  ✓ skintone_data/metadata
  ✓ skintone_data/labelled/train/dark
  ✓ skintone_data/labelled/train/light
  ✓ skintone_data/labelled/train/mid-dark
  ✓ skintone_data/labelled/train/mid-light
  ✓ skintone_data/labelled/val/dark
  ✓ skintone_data/labelled/val/light
  ✓ skintone_data/labelled/val/mid-dark
  ✓

# 🎯 ÉTAPE 2 : PRÉTRAITEMENT POUR MACHINE LEARNING CLASSIQUE





!pip install opencv-python scikit-image pandas numpy tqdm torchvision

In [2]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle
import warnings
warnings.filterwarnings('ignore')

class MLSkinTonePreprocessor:
    """Préprocesseur pour apprentissage automatique semi-supervisé sur données d'images de teinte de peau"""
    
    def __init__(self, root_dir='skintone_data', feature_type='hog', n_components=100):
        """
        Args:
            root_dir (str): Chemin vers le dossier skintone_data
            feature_type (str): Type de features ('hog', 'sift', 'color_hist', 'combined')
            n_components (int): Nombre de composantes pour PCA
        """
        self.root_dir = Path(root_dir)
        self.feature_type = feature_type
        self.n_components = n_components
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=n_components, random_state=42)
        
        # Classes
        self.classes = ['dark', 'light', 'mid-dark', 'mid-light']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.idx_to_class = {idx: cls for cls, idx in self.class_to_idx.items()}
        
        # Paramètres pour extraction de features
        self.target_size = (128, 128)  # Taille pour redimensionnement
    
    def extract_hog_features(self, image):
        """Extrait les features HOG d'une image"""
        # Convertir en niveaux de gris
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # Calculer HOG
        hog = cv2.HOGDescriptor()
        features = hog.compute(gray)
        
        return features.flatten()
    
    def extract_sift_features(self, image):
        """Extrait les features SIFT d'une image"""
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # Initialiser SIFT
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        if descriptors is None:
            # Si pas de keypoints, retourner un vecteur nul
            return np.zeros(128)  # SIFT produit 128 dimensions par défaut
        
        # Utiliser la moyenne des descripteurs
        return np.mean(descriptors, axis=0)
    
    def extract_color_histogram(self, image, bins=32):
        """Extrait l'histogramme de couleur d'une image"""
        # Calculer histogramme pour chaque canal
        hist_r = cv2.calcHist([image], [0], None, [bins], [0, 256])
        hist_g = cv2.calcHist([image], [1], None, [bins], [0, 256])
        hist_b = cv2.calcHist([image], [2], None, [bins], [0, 256])
        
        # Concaténer et normaliser
        hist = np.concatenate([hist_r, hist_g, hist_b]).flatten()
        hist = cv2.normalize(hist, hist).flatten()
        
        return hist
    
    def extract_combined_features(self, image):
        """Extrait des features combinées (HOG + histogramme de couleur)"""
        hog_features = self.extract_hog_features(image)
        color_features = self.extract_color_histogram(image)
        
        return np.concatenate([hog_features, color_features])
    
    def extract_features(self, image_path):
        """Extrait les features d'une image selon le type choisi"""
        # Charger et redimensionner l'image
        image = Image.open(image_path).convert('RGB')
        image = np.array(image)
        image = cv2.resize(image, self.target_size)
        
        if self.feature_type == 'hog':
            return self.extract_hog_features(image)
        elif self.feature_type == 'sift':
            return self.extract_sift_features(image)
        elif self.feature_type == 'color_hist':
            return self.extract_color_histogram(image)
        elif self.feature_type == 'combined':
            return self.extract_combined_features(image)
        else:
            raise ValueError(f"Type de feature non supporté: {self.feature_type}")
    
    def load_labelled_data(self, split='train'):
        """Charge et traite les données labellisées"""
        print(f"📥 Chargement des données labellisées ({split})...")
        
        features_list = []
        labels_list = []
        filenames = []
        
        split_dir = self.root_dir / 'labelled' / split
        
        for cls in self.classes:
            class_dir = split_dir / cls
            if class_dir.exists():
                print(f"  Traitement classe {cls}...")
                
                for img_path in class_dir.glob('*.jpg'):
                    try:
                        features = self.extract_features(img_path)
                        features_list.append(features)
                        labels_list.append(self.class_to_idx[cls])
                        filenames.append(img_path.name)
                    except Exception as e:
                        print(f"    ⚠️ Erreur avec {img_path.name}: {e}")
                
                for img_path in class_dir.glob('*.jpeg'):
                    try:
                        features = self.extract_features(img_path)
                        features_list.append(features)
                        labels_list.append(self.class_to_idx[cls])
                        filenames.append(img_path.name)
                    except Exception as e:
                        print(f"    ⚠️ Erreur avec {img_path.name}: {e}")
                
                for img_path in class_dir.glob('*.png'):
                    try:
                        features = self.extract_features(img_path)
                        features_list.append(features)
                        labels_list.append(self.class_to_idx[cls])
                        filenames.append(img_path.name)
                    except Exception as e:
                        print(f"    ⚠️ Erreur avec {img_path.name}: {e}")
        
        X = np.array(features_list)
        y = np.array(labels_list)
        
        print(f"  ✓ {len(X)} échantillons chargés")
        
        return X, y, filenames
    
    def load_unlabelled_data(self):
        """Charge et traite les données non-labellisées"""
        print("📥 Chargement des données non-labellisées...")
        
        features_list = []
        filenames = []
        original_classes = []
        
        unlabelled_dir = self.root_dir / 'unlabelled'
        mapping_file = self.root_dir / 'metadata' / 'unlabelled_mapping.csv'
        
        # Charger le mapping si disponible
        mapping_dict = {}
        if mapping_file.exists():
            mapping_df = pd.read_csv(mapping_file)
            mapping_dict = dict(zip(mapping_df['filename'], mapping_df['original_class']))
        
        if unlabelled_dir.exists():
            for img_path in unlabelled_dir.glob('*.jpg'):
                try:
                    features = self.extract_features(img_path)
                    features_list.append(features)
                    filenames.append(img_path.name)
                    original_classes.append(mapping_dict.get(img_path.name, 'unknown'))
                except Exception as e:
                    print(f"    ⚠️ Erreur avec {img_path.name}: {e}")
            
            for img_path in unlabelled_dir.glob('*.jpeg'):
                try:
                    features = self.extract_features(img_path)
                    features_list.append(features)
                    filenames.append(img_path.name)
                    original_classes.append(mapping_dict.get(img_path.name, 'unknown'))
                except Exception as e:
                    print(f"    ⚠️ Erreur avec {img_path.name}: {e}")
            
            for img_path in unlabelled_dir.glob('*.png'):
                try:
                    features = self.extract_features(img_path)
                    features_list.append(features)
                    filenames.append(img_path.name)
                    original_classes.append(mapping_dict.get(img_path.name, 'unknown'))
                except Exception as e:
                    print(f"    ⚠️ Erreur avec {img_path.name}: {e}")
        
        X = np.array(features_list)
        
        print(f"  ✓ {len(X)} échantillons non-labellisés chargés")
        
        return X, filenames, original_classes
    
    def preprocess_data(self):
        """Prétraite toutes les données et applique PCA + standardisation"""
        print("🔧 PRÉTRAITEMENT DES DONNÉES")
        print("=" * 60)
        
        # Charger les données
        X_train_labelled, y_train_labelled, train_filenames = self.load_labelled_data('train')
        X_val, y_val, val_filenames = self.load_labelled_data('val')
        X_test, y_test, test_filenames = self.load_labelled_data('test')
        X_unlabelled, unlabelled_filenames, unlabelled_classes = self.load_unlabelled_data()
        
        # Combiner toutes les features pour ajuster le scaler et PCA
        print("📊 Ajustement du scaler et PCA...")
        all_features = np.vstack([X_train_labelled, X_val, X_test, X_unlabelled])
        
        # Standardisation
        all_features_scaled = self.scaler.fit_transform(all_features)
        
        # PCA
        all_features_pca = self.pca.fit_transform(all_features_scaled)
        
        print(f"  Variance expliquée par PCA: {self.pca.explained_variance_ratio_[:5]} (top 5)")
        print(f"  Variance totale expliquée: {np.sum(self.pca.explained_variance_ratio_):.3f}")
        
        # Séparer les données transformées
        n_train = len(X_train_labelled)
        n_val = len(X_val)
        n_test = len(X_test)
        
        X_train_pca = all_features_pca[:n_train]
        X_val_pca = all_features_pca[n_train:n_train+n_val]
        X_test_pca = all_features_pca[n_train+n_val:n_train+n_val+n_test]
        X_unlabelled_pca = all_features_pca[n_train+n_val+n_test:]
        
        # Créer les datasets finaux
        datasets = {
            'train': {
                'X': X_train_pca,
                'y': y_train_labelled,
                'filenames': train_filenames
            },
            'val': {
                'X': X_val_pca,
                'y': y_val,
                'filenames': val_filenames
            },
            'test': {
                'X': X_test_pca,
                'y': y_test,
                'filenames': test_filenames
            },
            'unlabelled': {
                'X': X_unlabelled_pca,
                'filenames': unlabelled_filenames,
                'original_classes': unlabelled_classes
            }
        }
        
        print("✅ Prétraitement terminé!")
        
        return datasets
    
    def save_preprocessed_data(self, datasets, output_dir='preprocessed_ml_data'):
        """Sauvegarde les données prétraitées"""
        os.makedirs(output_dir, exist_ok=True)
        
        print(f"💾 Sauvegarde dans {output_dir}...")
        
        # Sauvegarder les datasets
        for split, data in datasets.items():
            np.savez_compressed(
                os.path.join(output_dir, f'{split}_data.npz'),
                X=data['X'],
                y=data.get('y', np.full(len(data['X']), -1)),  # -1 pour non-labellisé
                filenames=data['filenames']
            )
        
        # Sauvegarder le scaler et PCA
        with open(os.path.join(output_dir, 'scaler.pkl'), 'wb') as f:
            pickle.dump(self.scaler, f)
        
        with open(os.path.join(output_dir, 'pca.pkl'), 'wb') as f:
            pickle.dump(self.pca, f)
        
        # Sauvegarder les métadonnées
        metadata = {
            'feature_type': self.feature_type,
            'n_components': self.n_components,
            'classes': self.classes,
            'target_size': self.target_size,
            'splits_info': {split: len(data['X']) for split, data in datasets.items()}
        }
        
        with open(os.path.join(output_dir, 'metadata.pkl'), 'wb') as f:
            pickle.dump(metadata, f)
        
        print("  ✓ Données sauvegardées")
    
    def load_preprocessed_data(self, input_dir='preprocessed_ml_data'):
        """Charge les données prétraitées"""
        print(f"📂 Chargement des données depuis {input_dir}...")
        
        datasets = {}
        
        for split in ['train', 'val', 'test', 'unlabelled']:
            data = np.load(os.path.join(input_dir, f'{split}_data.npz'))
            datasets[split] = {
                'X': data['X'],
                'y': data['y'],
                'filenames': data['filenames']
            }
        
        # Charger scaler et PCA
        with open(os.path.join(input_dir, 'scaler.pkl'), 'rb') as f:
            self.scaler = pickle.load(f)
        
        with open(os.path.join(input_dir, 'pca.pkl'), 'rb') as f:
            self.pca = pickle.load(f)
        
        # Charger métadonnées
        with open(os.path.join(input_dir, 'metadata.pkl'), 'rb') as f:
            metadata = pickle.load(f)
        
        print("  ✓ Données chargées")
        
        return datasets, metadata

def main():
    """Fonction principale"""
    print("🎯 PRÉPROCESSING POUR APPRENTISSAGE AUTOMATIQUE SEMI-SUPERVISÉ")
    print("=" * 60)
    
    # Créer le préprocesseur
    preprocessor = MLSkinTonePreprocessor(
        root_dir='skintone_data',
        feature_type='combined',  # 'hog', 'sift', 'color_hist', 'combined'
        n_components=100
    )
    
    # Prétraite les données
    datasets = preprocessor.preprocess_data()
    
    # Afficher les statistiques
    print("\n📊 STATISTIQUES DES DONNÉES PRÉTRAITÉES:")
    for split, data in datasets.items():
        n_samples = len(data['X'])
        n_features = data['X'].shape[1]
        print(f"  {split}: {n_samples} échantillons, {n_features} features")
        if 'y' in data and split != 'unlabelled':
            unique_labels, counts = np.unique(data['y'], return_counts=True)
            print(f"    Classes: {dict(zip(unique_labels, counts))}")
    
    # Sauvegarder les données
    preprocessor.save_preprocessed_data(datasets, output_dir='preprocessed_ml_data')
    
    print("\n✅ Préprocessing terminé!")
    print("Vous pouvez maintenant utiliser ces données pour vos algorithmes ML semi-supervisés.")
    print("\n📁 Fichiers créés:")
    print("  - preprocessed_ml_data/train_data.npz")
    print("  - preprocessed_ml_data/val_data.npz")
    print("  - preprocessed_ml_data/test_data.npz")
    print("  - preprocessed_ml_data/unlabelled_data.npz")
    print("  - preprocessed_ml_data/scaler.pkl")
    print("  - preprocessed_ml_data/pca.pkl")
    print("  - preprocessed_ml_data/metadata.pkl")
    
    return datasets

if __name__ == "__main__":
    """
    INSTRUCTIONS:
    1. Assurez-vous que le dossier 'skintone_data' existe (après avoir exécuté le script de split)
    2. Exécutez ce code dans un notebook Jupyter
    3. Les données prétraitées seront sauvegardées dans 'preprocessed_ml_data/'
    
    TYPES DE FEATURES DISPONIBLES:
    - 'hog': Histogram of Oriented Gradients
    - 'sift': Scale-Invariant Feature Transform
    - 'color_hist': Histogramme de couleur
    - 'combined': HOG + histogramme de couleur
    
    UTILISATION POUR ALGORITHMES SEMI-SUPERVISÉS:
    - Pseudo-labeling: Utilisez train + unlabelled avec un classifieur de base
    - Graph-based: Utilisez sklearn.semi_supervised.LabelPropagation
    - Consistency regularization: Peut être adapté avec des méthodes d'ensemble
    """
    datasets = main()


🎯 PRÉPROCESSING POUR APPRENTISSAGE AUTOMATIQUE SEMI-SUPERVISÉ
🔧 PRÉTRAITEMENT DES DONNÉES
📥 Chargement des données labellisées (train)...
  Traitement classe dark...
  Traitement classe light...
  Traitement classe mid-dark...
  Traitement classe mid-light...
  ✓ 5748 échantillons chargés
📥 Chargement des données labellisées (val)...
  Traitement classe dark...
  Traitement classe light...
  Traitement classe mid-dark...
  Traitement classe mid-light...
  ✓ 1228 échantillons chargés
📥 Chargement des données labellisées (test)...
  Traitement classe dark...
  Traitement classe light...
  Traitement classe mid-dark...
  Traitement classe mid-light...
  ✓ 1236 échantillons chargés
📥 Chargement des données non-labellisées...
  ✓ 19164 échantillons non-labellisés chargés
📊 Ajustement du scaler et PCA...
  Variance expliquée par PCA: [0.05828715 0.03958944 0.02882378 0.02233959 0.02217248] (top 5)
  Variance totale expliquée: 0.539
✅ Prétraitement terminé!

📊 STATISTIQUES DES DONNÉES PRÉTRAI